In [1]:
import os
import pandas as pd
import random
import sys
sys.path.append(os.path.join(os.getcwd(), "birdmaps"))

In [3]:
import ebird_db, bird_runs

In [ ]:
# This needs to point to your eBird database. 
ebird_obs = birdmaps.ebird_db.EbirdObservations("/Users/luca/data/eBird/bird_data_uswest.db")

In [ ]:
valid = ebird_db.Validation()
FILE_PATH = "data/CA-Final"
bird_run = bird_runs.BirdRun(FILE_PATH)

In [ ]:
birds = [
    bird_run.get_bird_run("acowoo", "Acorn Woodpecker"),
    bird_run.get_bird_run("stejay", "Steller's Jay"),
    ]

max_distances = [2]
date_range = ("2012-01-01", "2018-12-31")
breeding = True
state = "US-CA"
num_sample_squares = 20000 # Sampling number for the squares.

In [ ]:

for bird in birds:
    print("Bird:", bird.name)
    for max_dist in max_distances:
        print("Max distance:", max_dist)
        bird_run.createdir(bird.obs_path)
        df = pd.DataFrame(columns=["Square", "NumChecklists", "NumBirdChecklists", "NumBirds"])
        squares = ebird_obs.get_all_squares(state=state, breeding=breeding,
                                            date_range=date_range, max_dist=max_dist)
        num_squares = len(squares)
        k = min(num_squares, num_sample_squares)
        print("Retrieved", num_squares, "squares")
        # print(squares[:10])
        selected_squares = random.sample(squares, k)
        for i, sq in enumerate(selected_squares):
            num_checklists, num_bird_checklists, num_birds = ebird_obs.get_square_observations(
                sq, bird, breeding=breeding, date_range=date_range, max_dist=max_dist)
            df = df.append({
                "Square": sq, "NumChecklists": num_checklists,
                "NumBirdChecklists": num_bird_checklists, "NumBirds": num_birds
            }, ignore_index=True)
            if (i + 1) % 1000 == 0:
                print("Processed", i + 1, "squares")
        obs_fn = bird_run.get_observations_all_fn(bird.obs_path, max_distance=max_dist,
                                                  date_range="-".join(date_range),
                                                  num_squares=num_sample_squares)
        df.to_csv(obs_fn)